In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
dataset = pd.read_csv("hour.csv")

---

Exploratory Data Analysis

In [3]:
# Display first 5 records 
dataset.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [ ]:
# Info about columns, type of data 
dataset.info()

In [ ]:
# Getting various statistical data like Mean, Standard Deviation, Median, Max Value, Min Value (excludes categorical vars)
dataset.describe()

In [ ]:
# Checking if columns have null values, if null- we replace with mean/median or simply drop these entries
dataset.isnull().sum()

In [ ]:
# Get the count of zeros in column 'cnt'  
column = dataset['cnt'] 
count = column[column == 0].count()
print('Count of zeros in cnt : ', count)

In [ ]:
# Checking correlation between variables (including target var to identify how features affect it) 
dataset.corr()

---

Univariate Analysis

In [ ]:
# Relationship between Temperation and Count
sns.lineplot(x='temp', y='cnt', data=dataset).set(xlabel="Temperature", ylabel="Total Bikes Rented")

In [ ]:
# Relationship between Humidity and Count
sns.lineplot(x='hum', y='cnt', data=dataset).set(xlabel="Humidity", ylabel="Total Bikes Rented")

In [ ]:
# Count of bikes rented per season
sns.countplot(x='season', data=dataset).set(xlabel="Season", ylabel="Total Bikes Rented")
dataset['season'].value_counts()

In [ ]:
# Count of bikes rented per year(2011, 2012)
sns.violinplot(x='yr', y='cnt', data=dataset).set(xlabel="Year", ylabel="Total Bikes Rented")
dataset['yr'].value_counts()

In [ ]:
# Count of bikes rented per month
plt.figure(figsize = (12,8))
sns.barplot(x='mnth', y='cnt', data=dataset).set(xlabel="Month", ylabel="Total Bikes Rented")

---

Bivariate Analysis

In [ ]:
# Bikes Rented vs Weather Condition with respect to the year
sns.barplot(x="weathersit", y="cnt", hue="yr", data=dataset).set(xlabel="Weather Condition", ylabel="Total Bikes Rented")
# how to get count of bikes in each category and year??

In [ ]:
# Bikes Rented vs Month with respect to the season
plt.figure(figsize = (12,8))
sns.barplot(x="workingday", y="cnt", hue="season", data=dataset).set(xlabel="Working Day", ylabel="Total Bikes Rented")

In [ ]:
# Bikes rented per hour with respect to a season
plt.figure(figsize = (12,8))
sns.pointplot(x="hr", y="cnt", hue="season", data=dataset).set(xlabel="Hour", ylabel="Total Bikes Rented")

- Peaks can be seen at around 8am and 5pm (office hours)

---

Outlier Analysis

In [ ]:
# Outlier Analysis for month
plt.figure(figsize = (20,8))
sns.boxplot(data=dataset[['mnth', 'cnt']], x='mnth', y='cnt').set(xlabel="Month", ylabel="Total Bikes Rented")

In [ ]:
# Outlier Analysis for hours
plt.figure(figsize = (20,8))
sns.boxplot(data=dataset[['hr', 'cnt']], x='hr', y='cnt').set(xlabel="Hour", ylabel="Total Bikes Rented")

In [ ]:
# Outlier Analysis for cnt
plt.figure(figsize = (8,6))
sns.boxplot(data=dataset[['cnt']], y='cnt').set(ylabel="Total Bikes Rented")

In [ ]:
# calculate interquartile range
q25, q75 = np.percentile(dataset['cnt'], 25), np.percentile(dataset['cnt'], 75)
iqr = q75 - q25
print("Interquartile Range", iqr)

# calculate the outlier cutoff
cut_off = iqr * 1.5
lower = q25 - cut_off
upper = 600

In [ ]:
# identify outliers
outliers = [x for x in dataset['cnt'] if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))

In [ ]:
dataset[(dataset['cnt'] < lower) | (dataset['cnt'] > upper)]

In [ ]:
# Removing the Outliers
dataset = dataset[(dataset['cnt'] > lower) & (dataset['cnt'] < upper)]

dataset.head()

In [ ]:
len(dataset.index)

In [ ]:
# Outlier Analysis for cnt
plt.figure(figsize = (8,6))
sns.boxplot(data=dataset[['cnt']], y='cnt').set(ylabel="Total Bikes Rented")

---

Feature Transformation

In [ ]:
# Before dropping date, let us introduce a days_old variable which indicates how old is the business
dataset['days_old'] = (pd.to_datetime(dataset['dteday'],format= '%Y-%m-%d') - pd.to_datetime('2011-01-01',format= '%Y-%m-%d')).dt.days

In [ ]:
dataset.tail()

In [ ]:
# Dropping instant, dteday, casual, registered
dataset = dataset.drop(['instant', 'dteday', 'casual', 'registered'], axis=1)

In [ ]:
# Convert season, weathersit, weekday and month to categorical data
dataset.season = dataset.season.replace({1:"spring", 2:"summer", 3:"fall", 4:"winter"})

dataset.weathersit = dataset.weathersit.replace({1:'good',2:'moderate',3:'bad',4:'severe'})

dataset.mnth = dataset.mnth.replace({1: 'jan',2: 'feb',3: 'mar',4: 'apr',5: 'may',6: 'jun',
                  7: 'jul',8: 'aug',9: 'sept',10: 'oct',11: 'nov',12: 'dec'})

dataset.weekday = dataset.weekday.replace({0: 'sun',1: 'mon',2: 'tue',3: 'wed',4: 'thu',5: 'fri',6: 'sat'})

In [ ]:
dataset.head()

In [ ]:
dataset.tail()

In [ ]:
# One-hot encoding season, mnth, weekday, weathersit

dataset = pd.get_dummies(data=dataset,columns=["season","mnth","weekday", "weathersit"])

In [ ]:
dataset.head()

In [ ]:
dataset.tail()

---

Model Development

In [ ]:
working_dt = dataset

In [ ]:
y = working_dt.pop('cnt')
y.head()

In [ ]:
X = working_dt
X.head()

In [ ]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
y_train.head()

In [ ]:
y_test.head()

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Function to build OLS model with columns as an input 

def build_ols_model(cols):
    # adding the constant term
    X_const = sm.add_constant(X_train[cols])
    # performing the regression and fitting the model
    result = sm.OLS(y_train, X_const).fit()
    # printing result
    print(result.summary())

In [ ]:
# Function to plot color scaled correlation matrix (Checking Multicollinearity)
def compute_correlation(cols):
    plt.figure(figsize = (15,10))
    corr = X[cols].corr()
    sns.heatmap(corr, cmap='GnBu', annot=True)
    plt.show()

In [ ]:
# Function Compute VIF data for each independent variable in the X_const set (Checking Multicollinearity)

def compute_vif(cols):
    vif = pd.DataFrame()
    vif["features"] = X_train[cols].columns
    vif["vif_Factor"] = [variance_inflation_factor(X_train[cols].values, i) for i in range(X_train[cols].shape[1])]
    print(vif.sort_values(by='vif_Factor', ascending = False))

---

Model Number 1: Taking all independent features



In [ ]:
cols = X_train.columns
print(cols)

In [ ]:
# Build OLS model 1
build_ols_model(cols)

In [ ]:
# Compute correlation matrix for model 1
compute_correlation(cols)

In [ ]:
# Compute Vif for model 1
compute_vif(cols)

---

In [ ]:
# Using RFE (Recursive Feature Extraction) to select best 15 features

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

lr = LinearRegression()
lr.fit(X_train, y_train)

rfe = RFE(lr, 21)
rfe.fit(X_train, y_train)

In [ ]:
# Features selected by RFE
X_train.columns[rfe.support_]

---

Model 2: Taking features selected by RFE

In [ ]:
cols = X_train.columns[rfe.support_]
print(cols)

In [ ]:
# Build OLS model 2
build_ols_model(cols)

In [ ]:
# Compute correlation matrix for model 2
compute_correlation(cols)

In [ ]:
# Compute Vif for model 2
compute_vif(cols)

---

Model 3: Dropping atemp

In [ ]:
cols = ['yr', 'holiday', 'temp', 'hum', 'windspeed', 'season_spring',
       'season_winter', 'mnth_aug', 'mnth_dec', 'mnth_feb', 'mnth_jan',
       'mnth_jul', 'mnth_jun', 'weathersit_severe']
print(cols)

In [ ]:
# Build OLS model 3
build_ols_model(cols)

In [ ]:
# Compute correlation matrix for model 3
compute_correlation(cols)

In [ ]:
# Compute Vif for model 3
compute_vif(cols)

---

Model 4: Dropping weathersit_severe

In [ ]:
cols = ['yr', 'holiday', 'temp', 'hum', 'windspeed', 'season_spring',
       'season_winter', 'mnth_aug', 'mnth_dec', 'mnth_feb', 'mnth_jan',
       'mnth_jul', 'mnth_jun']
print(cols)

In [ ]:
# Build OLS model 4
build_ols_model(cols)

In [ ]:
# Compute correlation matrix for model 4
compute_correlation(cols)

In [ ]:
# Compute Vif for model 4
compute_vif(cols)

---

Model 5: Including workingday

In [ ]:
cols = ['yr', 'holiday', 'temp', 'hum', 'windspeed', 'season_spring',
       'season_winter', 'mnth_aug', 'mnth_dec', 'mnth_feb', 'mnth_jan',
       'mnth_jul', 'mnth_jun', 'weathersit_severe', 'workingday']
print(cols)

In [ ]:
# Build OLS model 5
build_ols_model(cols)

In [ ]:
# Compute correlation matrix for model 5
compute_correlation(cols)

In [ ]:
# Compute Vif for model 5
compute_vif(cols)

---

Model 6: Including days_old

In [ ]:
cols = ['yr', 'holiday', 'temp', 'hum', 'windspeed', 'season_spring',
       'season_winter', 'mnth_aug', 'mnth_dec', 'mnth_feb', 'mnth_jan',
       'mnth_jul', 'mnth_jun', 'weathersit_severe', 'workingday', 'days_old']
print(cols)

In [ ]:
# Build OLS model 6
build_ols_model(cols)

In [ ]:
# Compute correlation matrix for model 6
compute_correlation(cols)

In [ ]:
# Compute Vif for model 6
compute_vif(cols)

---

Drawbacks of each model:

In [ ]:
# Model 1,2,6- Multicollinearity
# Model 3,5- Presence of workinday which has a P|t|>0.05 which shows the irrelevance of the feature to the target variable
# Therefore, we will choose Model 4 goiong forward

In [ ]:
cols = ['yr', 'holiday', 'temp', 'hum', 'windspeed', 'season_spring',
       'season_winter', 'mnth_aug', 'mnth_dec', 'mnth_feb', 'mnth_jan',
       'mnth_jul', 'mnth_jun']
print(cols)

In [ ]:
lr1 = LinearRegression()
reg_result = lr1.fit(X_train[cols],y_train)
print(reg_result.intercept_, reg_result.coef_)

---

Model Evaluation

In [ ]:
working_dt['cnt'] = y
working_dt.head()

- Linearity

In [ ]:
# visualize the relationship between the features and the response using scatterplots
p = sns.pairplot(dataset, x_vars=['yr', 'holiday', 'temp', 'hum', 'windspeed', 'season_spring',
       'season_winter', 'mnth_aug', 'mnth_dec', 'mnth_feb', 'mnth_jan',
       'mnth_jul', 'mnth_jun'], y_vars='cnt', size=10, aspect=0.7)

- Mean of Residuals

In [ ]:
y_train_pred = lr1.predict(X_train[cols])

In [ ]:
residuals = y_train.values-y_train_pred
mean_residuals = np.mean(residuals)
print("Mean of Residuals {}".format(mean_residuals))

- Check for Homoscedasticity

In [ ]:
p = sns.scatterplot(y_train_pred,residuals)
plt.xlabel('Predicted values of y_train')
plt.ylabel('Residuals')
plt.ylim(-10,10)
plt.xlim(0,26)
p = sns.lineplot([0,26],[0,0],color='blue')
p = plt.title('Residuals vs fitted values plot for homoscedasticity check')

In [ ]:
# Checking heteroscedasticity : Using Goldfeld Quandt we test for heteroscedasticity.
# Null Hypothesis: Error terms are homoscedastic
# Alternative Hypothesis: Error terms are heteroscedastic.

# from statsmodels.compat import lzip

# name = ['F statistic', 'p-value']
# test = sm.het_goldfeldquandt(residuals, X_train)
# lzip(name, test)

- Normality of Errors

In [ ]:
p = sns.distplot(residuals,kde=True)
p = plt.title('Normality of error terms/residuals')

- No autocorrelation of residuals

In [ ]:
plt.figure(figsize=(10,5))
p = sns.lineplot(y_train_pred,residuals,marker='o',color='blue')
plt.xlabel('Predicted values of y_train')
plt.ylabel('Residuals')
plt.ylim(-100,100)
plt.xlim(0,26)
p = sns.lineplot([0,26],[0,0],color='red')
p = plt.title('Residuals vs fitted values plot for autocorrelation check')

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
#Print R-squared Value
r2_score(y_train,y_train_pred)

In [ ]:
#Predict the values for test data
y_test_pred = lr1.predict(X_test[cols])

In [ ]:
# Find out the R squared value between test and predicted test data sets.  
r2_score(y_test,y_test_pred)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mse = mean_squared_error(y_test, y_test_pred)

In [ ]:
import math

rmse = math.sqrt(mse)
print(rmse)